In [1]:
import pandas as pd
import numpy as np
import re
import string

from nltk.corpus import stopwords
import nltk
import glob
from  nltk import word_tokenize

from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.corpus.reader.plaintext import PlaintextCorpusReader
import os
from collections import Counter
from collections import defaultdict
import gc
from gensim.models import FastText
import pandas as pd


In [3]:
!wget https://www.dropbox.com/s/aogtja9upycdqv4/hpac_splits.zip?dl=0

--2022-11-17 17:38:29--  https://www.dropbox.com/s/aogtja9upycdqv4/hpac_splits.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/aogtja9upycdqv4/hpac_splits.zip [following]
--2022-11-17 17:38:29--  https://www.dropbox.com/s/raw/aogtja9upycdqv4/hpac_splits.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc59d7682a7c6d02e06700082660.dl.dropboxusercontent.com/cd/0/inline/Bw-02CFnWHgGtr0du0A81fRw3HmD1OyHpQrddA1yLKEcuEnF6BzMFFSYe3fTJRNl26PZSsEZ2WACcChXNbFHYCVlY5hlSLVd-pkwb_qqTSANAQq9HXI7d8MCxpxEAcrBslYf36_qbkVx03Bp0W3BRhMzQfCKupM8Mg0cfUMcl0wPlg/file# [following]
--2022-11-17 17:38:30--  https://uc59d7682a7c6d02e06700082660.dl.dropboxusercontent.com/cd/0/inline/Bw-02CFnWHgGtr0du0A81fRw3HmD1OyHpQrddA1yLKEcuEnF6BzMFFSYe3fTJRN

In [4]:
!unzip hpac_splits.zip?dl=0

Archive:  hpac_splits.zip?dl=0
   creating: hpac_corpus/
  inflating: hpac_corpus/hpac_training_128.tsv  
   creating: __MACOSX/
   creating: __MACOSX/hpac_corpus/
  inflating: __MACOSX/hpac_corpus/._hpac_training_128.tsv  
  inflating: hpac_corpus/hpac_dev_128.tsv  
  inflating: __MACOSX/hpac_corpus/._hpac_dev_128.tsv  
  inflating: hpac_corpus/hpac_test_128.tsv  
  inflating: __MACOSX/hpac_corpus/._hpac_test_128.tsv  
  inflating: __MACOSX/._hpac_corpus  


In [2]:
train_df = pd.read_csv('hpac_corpus/hpac_training_128.tsv', sep = '\t', header = None)
val_df = pd.read_csv('hpac_corpus/hpac_dev_128.tsv', sep = '\t', header = None)
test_df = pd.read_csv('hpac_corpus/hpac_test_128.tsv', sep = '\t', header = None)


In [3]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
punct = set(list(string.punctuation))
stop = set(stopwords.words('english') + list(string.punctuation))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
def format_text(text):
    text = str(text).lower()
    stop_words = set(stopwords.words('english'))
    punct_sym = string.punctuation
    text = text.translate(str.maketrans("","",punct_sym))
    text = " ".join([w for w in text.split() if w not in stop_words])    
    return text 

In [6]:
%%time
train_df[2]=train_df[2].apply(lambda x: format_text(x))
val_df[2]=val_df[2].apply(lambda x: format_text(x))
test_df[2]=test_df[2].apply(lambda x: format_text(x))

CPU times: user 10.1 s, sys: 1 s, total: 11.1 s
Wall time: 12.3 s


In [7]:
train_df = train_df.iloc[:,1:]
train_df.columns = ['label','text']
train_df = train_df[['text','label']]

val_df = val_df.iloc[:,1:]
val_df.columns = ['label','text']
val_df = val_df[['text','label']]

test_df = test_df.iloc[:,1:]
test_df.columns = ['label','text']
test_df = test_df[['text','label']]


In [8]:
import torchtext
import spacy
from torchtext.legacy import data 


nlp = spacy.load("en_core_web_sm")
tokenizer = lambda sent: [x.text for x in nlp.tokenizer(sent) if x.text != " "]
        
TEXT = data.Field(sequential=True, tokenize=tokenizer, lower=True)
LABEL = data.LabelField()
datafields = [("text",TEXT),("label",LABEL)]

In [9]:
# https://gist.github.com/nissan/ccb0553edb6abafd20c3dec34ee8099d
class DataFrameDataset(torchtext.legacy.data.Dataset):

    def __init__(self, df, text_field, label_field, is_test=False, **kwargs):
        fields = [('text', text_field), ('label', label_field)]
        # examples = []
        # for i, row in tqdm(df.iterrows()):
        #     label = row.label if not is_test else None
        #     text = row.text
        #     examples.append(data.Example.fromlist([text, label], fields))

        # super().__init__(examples, fields, **kwargs)
        examples = [data.Example.fromlist(i, fields) for i in df.values.tolist()]
        tdata = data.Dataset(examples, fields)
        super().__init__(examples, fields, **kwargs)
    @staticmethod
    def sort_key(ex):
        return len(ex.text)

    @classmethod
    def splits(cls, text_field, label_field, train_df, val_df=None, test_df=None, **kwargs):
        train_data, val_data, test_data = (None, None, None)

        if train_df is not None:
            train_data = cls(train_df.copy(), text_field, label_field, **kwargs)
        if val_df is not None:
            val_data = cls(val_df.copy(), text_field, label_field, **kwargs)
        if test_df is not None:
            test_data = cls(test_df.copy(), text_field, label_field, True, **kwargs)

        return tuple(d for d in (train_data, val_data, test_data) if d is not None)
        
train_ds, val_ds, test_ds = DataFrameDataset.splits(
  text_field=TEXT, label_field=LABEL, train_df=train_df, val_df=val_df, test_df=test_df)

In [10]:
TEXT.build_vocab(train_ds, max_size=25000)
LABEL.build_vocab(train_ds)

In [69]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CustomCNN(nn.Module):
    def __init__(self,
                 n,
                 emb_dim,
                 input_channel,
                 output_channel,
                 num_classes):
        super().__init__()
        self.embs = nn.Embedding(n, emb_dim)
        self.conv1 = nn.Conv2d( input_channel, output_channel, kernel_size=(2, emb_dim))
            
        self.conv2 = nn.Conv2d( input_channel, output_channel, kernel_size=(3, emb_dim))

        self.conv3 = nn.Conv2d( input_channel, output_channel, kernel_size=(4, emb_dim))

        self.conv4 = nn.Conv2d( input_channel, output_channel, kernel_size=(5, emb_dim))

        self.dropout = nn.Dropout()
        self.linear = nn.Linear(3 * output_channel, num_classes)

    def forward(self, x):
        embeddings = self.embs(x.t()).unsqueeze(1)
        relu1 = F.relu(self.conv1(embeddings).squeeze(3))
        relu2 = F.relu(self.conv2(embeddings).squeeze(3))
        relu3 = F.relu(self.conv3(embeddings).squeeze(3))
        relu4 = F.relu(self.conv4(embeddings).squeeze(3))


        layer_1 = F.max_pool1d(relu1, relu1.size(2)).squeeze(2)
        layer_2 = F.max_pool1d(relu2, relu2.size(2)).squeeze(2)
        layer_3 = F.max_pool1d(relu3, relu3.size(2)).squeeze(2)
        layer_4 = F.max_pool1d(relu3, relu4.size(2)).squeeze(2)
        
        #print(f"{layer_1.shape} {layer_2.shape} {layer_3.shape} {layer_4.shape}")
        res = self.dropout(torch.cat((layer_1,layer_2,layer_3), 1))

        return self.linear(res)

In [70]:
model = CustomCNN(len(TEXT.vocab),100,1,50,len(LABEL.vocab))

In [71]:
[len(TEXT.vocab),100,1,200,len(LABEL.vocab)]

[25002, 100, 1, 200, 85]

In [72]:
%%time
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_ds, val_ds, test_ds), 
    batch_sizes=(32,32,32), 
    sort_key=lambda x: len(x.text), 
    repeat=False,
    shuffle=True)

CPU times: user 115 µs, sys: 3 µs, total: 118 µs
Wall time: 123 µs


In [102]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
model = model.cuda()

In [105]:
import time
def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0

    for batch in dataloader:
        optimizer.zero_grad()
        predicted_label = model(batch.text.cuda())
        loss = criterion(predicted_label.cpu(), batch.label)
        loss.backward()
        optimizer.step()
        total_acc += loss.item()
    return total_acc / len(dataloader)

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for batch in dataloader:
            predicted_label = model(batch.text.cuda())
            loss = criterion(predicted_label.cpu(), batch.label)
            total_acc += loss.item()
            total_count += batch.label.size(0)
    return total_acc/total_count



In [107]:
EPOCHS = 14
for epoch in tqdm(range(1, EPOCHS + 1)):
    epoch_start_time = time.time()
    train_loss = train(train_iterator)
    val_loss = evaluate(valid_iterator)

100%|██████████| 14/14 [04:27<00:00, 19.07s/it]


In [108]:
from sklearn.metrics import f1_score

accu_test = evaluate(test_iterator)
print('test f1 {:8.3f}'.format(accu_test))

test f1    0.120
